In [2]:
import numpy as np
import pandas as pd
import random
import os
import re # For regex in cleaning phone numbers

# Set the random seed for reproducibility
np.random.seed(42)

# Parameters
num_customers = 1000
num_stores = 20
start_date = pd.to_datetime('2025-01-01')
end_date = pd.to_datetime('2025-12-31')

# Sample names and emojis for non-ASCII characters
sample_names = ['Alice', 'Bob', 'Charlie', 'Dana', 'Eve', 'Frank', 'Grace', 'Heidi']
emojis = ['', ' 😊', ' ©', ' ö', ' ™', ' ®'] # More emojis/symbols

# Extended Phone number formats
phone_formats = [
    np.nan, '555-1234', '5551234', '555.1234', '(555) 123-4567',
    '+1 555 987 6543', '010-12345678', '999-ERROR', 'SHORT', 'LONG-NUMBER-TOO-LONG'
]

# Extended City names with inconsistencies
city_names = ['Cairo', 'Alexandria', 'Giza']
city_variations = {
    'Cairo': ['Cairo', 'Cario', 'Cai', 'Al-Qahira'],
    'Alexandria': ['Alexandria', 'Alex.', 'Alexandira', 'Al-Iskandariyah'],
    'Giza': ['Giza', 'Gizah', 'Jizah', 'Gizeh']
}

# 1. Generate customers table with more challenging issues
customers_data = []
# Generate base customers
for i in range(1, num_customers + 1):
    name = random.choice(sample_names) + random.choice(emojis)
    customers_data.append({
        'custID': i,
        'Customer Name': name,
        'PreferredStore': np.random.randint(1, num_stores + 1),
        'join_date': pd.to_datetime(random.choice(pd.date_range(start_date, end_date))),
        'Phone #': random.choice(phone_formats)
    })

customers = pd.DataFrame(customers_data)

# Introduce missing values in 'Customer Name'
customers.loc[customers.sample(frac=0.05).index, 'Customer Name'] = np.nan

# Introduce some fuzzy duplicates (same customer, slight name/phone variations)
num_fuzzy_duplicates = int(num_customers * 0.03) # 3% fuzzy duplicates
fuzzy_indices = customers.sample(n=num_fuzzy_duplicates, random_state=10).index
for idx in fuzzy_indices:
    original_row = customers.loc[idx].copy()
    new_cust_id = customers['custID'].max() + 1
    new_row = original_row.copy()
    new_row['custID'] = new_cust_id

    # Introduce variations
    if random.random() < 0.5: # 50% chance of name variation
        new_row['Customer Name'] = original_row['Customer Name'].replace('e', 'a', 1).replace('o', 'u', 1) if isinstance(original_row['Customer Name'], str) else 'Jane Doe'
    if random.random() < 0.5: # 50% chance of phone variation
        if isinstance(original_row['Phone #'], str) and '-' in original_row['Phone #']:
            new_row['Phone #'] = original_row['Phone #'].replace('-', '')
        elif isinstance(original_row['Phone #'], str) and ' ' in original_row['Phone #']:
            new_row['Phone #'] = original_row['Phone #'].replace(' ', '.')
        else:
            new_row['Phone #'] = random.choice(phone_formats) # new random format

    customers = pd.concat([customers, pd.DataFrame([new_row])], ignore_index=True)


# 2. Generate stores table with more challenging issues
stores_data = []
for i in range(1, num_stores + 1):
    original_city = random.choice(city_names)
    city_entry = random.choice(city_variations[original_city]) + random.choice(['', ' ']) # Add extra space sometimes
    stores_data.append({
        'STORE_id': i,
        'StoreName': f"Store_{i}{random.choice(['', '™', '§', ' Limited', ' Corp.'])}", # More variations
        'Size': np.random.choice(['Small', 'Medium', 'Large']),
        'City ': city_entry, # Use varied city entries
        'Opening': pd.to_datetime(random.choice(pd.date_range(start_date, end_date)))
    })
stores = pd.DataFrame(stores_data)

# Introduce inconsistent formats in the 'Size' column (some uppercase, some lowercase)
stores.loc[stores.sample(frac=0.10, random_state=1).index, 'Size'] = \
    stores.loc[stores.sample(frac=0.10, random_state=1).index, 'Size'].apply(lambda x: x.upper())
stores.loc[stores.sample(frac=0.05, random_state=2).index, 'Size'] = \
    stores.loc[stores.sample(frac=0.05, random_state=2).index, 'Size'].apply(lambda x: x.lower())


# 3. Generate transactions table with hierarchical and time-series data
num_transactions = 10000
payment_methods = ['Cash', 'Credit Card', 'Mobile Payment', 'Card', 'CC', 'Mobile', 'Unknown', np.nan] # More variations
transactions = pd.DataFrame({
    'TxnID': np.arange(1, num_transactions + 1),
    'custID': np.random.choice(customers['custID'], num_transactions),
    'STORE_id': np.random.choice(stores['STORE_id'], num_transactions),
    'TxnDate': pd.to_datetime(
        np.random.choice(
            pd.date_range(start_date, end_date, freq='D'),
            num_transactions
        )
    ),
    'Amount': np.round(np.random.uniform(5, 500, num_transactions), 2),
    'PaymentMethod': np.random.choice(payment_methods, num_transactions)
})

# Introduce missing values in 'PaymentMethod'
transactions.loc[transactions.sample(frac=0.05, random_state=2).index, 'PaymentMethod'] = np.nan

# Add inconsistent 'PaymentMethod' format (lowercase + trailing space)
transactions.loc[np.random.choice(transactions.index, size=100, replace=False), 'PaymentMethod'] = 'creditcard ' # trailing space
transactions.loc[np.random.choice(transactions.index, size=50, replace=False), 'PaymentMethod'] = 'cash ' # new trailing space


# 4. Introduce orphan transactions (invalid foreign keys) - Remains challenging
orphan_indices = transactions.sample(frac=0.02, random_state=3).index
transactions.loc[
    orphan_indices, 'custID'
] = customers['custID'].max() + np.random.randint(1, 50, size=len(orphan_indices)) # Use max + 1 to ensure truly new IDs
transactions.loc[
    orphan_indices, 'STORE_id'
] = stores['STORE_id'].max() + np.random.randint(1, 10, size=len(orphan_indices)) # Use max + 1


# 5. Introduce conflicting dates (TxnDate before join_date AND TxnDate before store Opening)
conflict_indices_cust = transactions.sample(frac=0.02, random_state=4).index
for idx in conflict_indices_cust:
    cust_id = transactions.at[idx, 'custID']
    join_date_series = customers.loc[customers['custID'] == cust_id, 'join_date']
    if not join_date_series.empty:
        transactions.at[idx, 'TxnDate'] = join_date_series.iloc[0] - pd.Timedelta(days=random.randint(15, 60))
    else:
        transactions.at[idx, 'TxnDate'] = start_date - pd.Timedelta(days=random.randint(30, 90))

# Conflict: Transaction before store opening date
conflict_indices_store = transactions.sample(frac=0.01, random_state=5).index # 1% additional conflicts
for idx in conflict_indices_store:
    store_id = transactions.at[idx, 'STORE_id']
    opening_date_series = stores.loc[stores['STORE_id'] == store_id, 'Opening']
    if not opening_date_series.empty:
        transactions.at[idx, 'TxnDate'] = opening_date_series.iloc[0] - pd.Timedelta(days=random.randint(5, 30))
    else:
        # If the transaction's STORE_id is orphaned, set TxnDate to a date earlier than start_date
        transactions.at[idx, 'TxnDate'] = start_date - pd.Timedelta(days=random.randint(40, 100))


# 6. Add messy 'Amount' formats for some transactions (mix numbers, strings with currency, other text)
amount_indices = transactions.sample(frac=0.07, random_state=6).index # Increased percentage
for idx in amount_indices:
    original_amount = transactions.at[idx, 'Amount']
    choice = random.random()
    if choice < 0.4:
        transactions.at[idx, 'Amount'] = f"${original_amount:.2f}"
    elif choice < 0.6:
        transactions.at[idx, 'Amount'] = f"€{original_amount:.2f}" # Euro
    elif choice < 0.7:
        transactions.at[idx, 'Amount'] = f"£{original_amount:.2f}" # Pound
    elif choice < 0.8:
        transactions.at[idx, 'Amount'] = "FREE" # Textual amount
    elif choice < 0.9:
        transactions.at[idx, 'Amount'] = "ZERO"
    else:
        transactions.at[idx, 'Amount'] = "N/A" # Another text type


# Create the base 'advanced_data' folder
base_output_folder = 'advanced_data_v2'
os.makedirs(base_output_folder, exist_ok=True)

# 7. Split transactions by month and save each file under 'advanced_data/monthly_transactions_v2'
monthly_transactions_folder = os.path.join(base_output_folder, 'monthly_transactions_v2')
os.makedirs(monthly_transactions_folder, exist_ok=True)
for month, group in transactions.groupby(transactions['TxnDate'].dt.to_period('M')):
    filename = f"{monthly_transactions_folder}/transactions_{month}.csv"
    group.to_csv(filename, index=False, encoding='utf-8')

# 8. Split customers by PreferredStore and save each file under 'advanced_data/customers_by_store_v2'
customers_by_store_folder = os.path.join(base_output_folder, 'customers_by_store_v2')
os.makedirs(customers_by_store_folder, exist_ok=True)
for store_id, group in customers.groupby('PreferredStore'):
    filename = f"{customers_by_store_folder}/customers_store_{store_id}.csv"
    group.to_csv(filename, index=False, encoding='utf-8')

# 9. Split stores by City and save each file under 'advanced_data/stores_by_city_v2'
# Clean the 'City ' column name for grouping (temporary for grouping, analyst needs to handle this in real cleaning)
# Note: The raw city names are still messy in the saved files
stores_for_city_split = stores.copy()
stores_for_city_split['City_Cleaned'] = stores_for_city_split['City '].str.strip().apply(
    lambda x: next((k for k, v in city_variations.items() if x in v), x)
)
stores_by_city_folder = os.path.join(base_output_folder, 'stores_by_city_v2')
os.makedirs(stores_by_city_folder, exist_ok=True)
for city_cleaned, group in stores_for_city_split.groupby('City_Cleaned'):
    sanitized_city = re.sub(r'[^a-zA-Z0-9_]', '', city_cleaned.replace(" ", "_")) # More robust sanitization
    filename = f"{stores_by_city_folder}/stores_{sanitized_city}.csv"
    # Save the original 'City ' column, not the temporary cleaned one
    group.drop(columns=['City_Cleaned']).to_csv(filename, index=False, encoding='utf-8')

print("✅ Generated EXTREMELY challenging advanced datasets with messy features and multiple split files; files are in 'advanced_data/'.")

✅ Generated EXTREMELY challenging advanced datasets with messy features and multiple split files; files are in 'advanced_data/'.


C:\Users\zzppo\AppData\Local\Temp\ipykernel_20852\436884651.py:159: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '$477.19' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  transactions.at[idx, 'Amount'] = f"${original_amount:.2f}"
